# **Mounting DRIVE and Installing Dependencies**



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! pip install openai
! pip install -U openai-whisper
! sudo apt update && sudo apt install ffmpeg
! pip install setuptools-rust
! pip install ffmpeg-python
! apt install ffmpeg
! apt install imagemagick
! pip install moviepy==2.0.0.dev2
! pip install imageio==2.25.1
! pip install pysrt==1.1.2
! cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 12.5 MB

# **ALL IMPORTS**

In [ ]:
import os
import requests
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip
from datetime import timedelta
import whisper
import json
# import ffmpeg
from google.colab import files
import imageio_ffmpeg as ffmpeg
import sys
import pysrt
import io
from PIL import ImageFont

downloadedVideosDirectory = "/content/drive/MyDrive/greWords/downloadedVideos"
outputVideosDirectory = "/content/drive/MyDrive/greWords/mergedVideos"

greWords = "/content/drive/MyDrive/greWords/greWords.json"

videoExtension = ".mp4"
audioExtension = ".mp3"
subtitleExtension = ".srt"

# **Convert MP4 to MP3**

In [ ]:
def getAllVideoFiles(currentWord):
    wordDirectory = downloadedVideosDirectory +"/"+ currentWord
    myFiles = [file for file in os.listdir(wordDirectory) if file.endswith(videoExtension)]
    return myFiles

def convertingFilesToAudio(videoLocation, audioLocation):
    video = VideoFileClip(videoLocation)
    video.audio.write_audiofile(audioLocation)

# convertingFilesToAudio("brother", getAllVideoFiles("brother"))

# **Transcribe the Audio**

In [ ]:
def transcribeAudio(audioLocation, subtitleLocation):
    model = whisper.load_model("small")
    print("Whisper model loaded.")
    transcribe = model.transcribe(audio=audioLocation)
    segments = transcribe['segments']

    for segment in segments:
        startTime = str(0)+str(timedelta(seconds=int(segment['start'])))+',000'
        endTime = str(0)+str(timedelta(seconds=int(segment['end'])))+',000'
        text = segment['text']
        segmentId = segment['id']+1
        segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] is ' ' else text}\n\n"

        with open(subtitleLocation, 'a+', encoding='utf-8') as srtFile:
            srtFile.write(segment)

<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-7-d320517d6435>:12: SyntaxWarning: "is" with a literal. Did you mean "=="?
  segment = f"{segmentId}\n{startTime} --> {endTime}\n{text[1:] if text[0] is ' ' else text}\n\n"


# **Make MP3 and SRT  (EXECUTION BATCH)**

In [ ]:
with open(greWords, "r") as file:
    data = json.load(file)

for currentWord, wordData in data.items():
    if wordData["clipsFound"] > 0 and currentWord != "brother":
        currentFileDirectory = downloadedVideosDirectory +"/"+ currentWord
        videoFiles = getAllVideoFiles(currentWord)
        print(videoFiles)
        videoClips = []
        for videoFile in videoFiles:
            fileName = videoFile.replace(videoExtension, "")
            videoLocation = currentFileDirectory +"/"+ fileName + videoExtension
            audioLocation = currentFileDirectory +"/"+ fileName + audioExtension
            subtitleLocation = currentFileDirectory +"/"+ fileName + subtitleExtension
            # print(videoLocation, audioLocation, subtitleLocation)
            clipDuration = VideoFileClip(videoLocation).duration
            print(clipDuration)
            with open(subtitleLocation, mode="w", encoding="utf-8") as subtitleFile:
                currentSubtitle = wordData["clipData"][fileName]["subtitle"]
                formatted_number = "{:02d}".format(int(clipDuration))
                subtitleData = f"1\n00:00:00,000 --> 00:00:{formatted_number},000\n{currentSubtitle}"
                print(subtitleData)
                subtitleFile.write(subtitleData)


['1.mp4', '2.mp4', '3.mp4', '4.mp4']
7.34
1
00:00:00,000 --> 00:00:07,000
There lives within the very flame of love... a kind of wick or snuff that will abate it.
3.18
1
00:00:00,000 --> 00:00:03,000
Consumes it with a burning flame that does not abate.
5.74
1
00:00:00,000 --> 00:00:05,000
You basically want to try to either aggravate the demon or abate it.
8.84
1
00:00:00,000 --> 00:00:08,000
l have unleashed a raging shit storm of epic proportions on the board of trustees of that pissant school that will not abate until those girls seek enrollment elsewhere.


# **Adding Subtitles to Video**

In [86]:
def time_to_seconds(time_obj):
    return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000

def createSubtitleClips(currentWord, subtitleLocation, videoSize):
    subtitles = pysrt.open(subtitleLocation)
    subtitle_clips = []
    fontsize = 30
    font = downloadedVideosDirectory + '/bangers.ttf'
    color = 'yellow'

    for subtitle in subtitles:
        start_time = time_to_seconds(subtitle.start)
        end_time = time_to_seconds(subtitle.end)
        duration = end_time - start_time
        # style = f'<span foreground="red" background="black" fontfamily="Bangers-Regular"><tt><i>{currentWord.upper()}</i></tt></span>!'
        # newSubtitle = subtitle.text.upper().replace(currentWord.upper(), style)
        newSubtitle = subtitle.text.upper()
        # print(font)
        print(newSubtitle)
        print(currentWord)
        # break
        text_clip = TextClip(
            newSubtitle,
            fontsize=fontsize,
            font=font,
            color=color,
            bg_color='none',  # Set your desired background color
            method='pango',
            size=(videoSize[0] * 3 / 4, None),
        ).set_start(start_time).set_duration(duration)

        subtitleX_Position = 'center'
        subtitleY_Position = videoSize[1] * 4 / 5
        textPosition = (subtitleX_Position, subtitleY_Position)

        subtitle_clips.append(text_clip.set_position(textPosition))

    return subtitle_clips


for currentWord, wordData in data.items():
    if wordData["clipsFound"] > 0:
        currentFileDirectory = downloadedVideosDirectory +"/"+ currentWord
        videoFiles = getAllVideoFiles(currentWord)
        print(videoFiles)
        outputLocation = outputVideosDirectory +"/"+ currentWord
        if not os.path.exists(outputLocation):
            os.makedirs(outputLocation)
        for videoFile in videoFiles:
            if currentWord != "brother":
                fileName = videoFile.replace(videoExtension, "")
                videoLocation = currentFileDirectory + "/" + fileName + videoExtension
                subtitleLocation = currentFileDirectory + "/" + fileName + subtitleExtension
                # outputLocation = currentFileDirectory + "/final" + fileName + videoExtension
                outputLocation = outputVideosDirectory +"/"+ currentWord +"/"+ fileName + videoExtension
                video = VideoFileClip(videoLocation)
                subtitle_clips = createSubtitleClips(currentWord, subtitleLocation, video.size)
                finalVideo = CompositeVideoClip([video] + subtitle_clips)
                finalVideo.write_videofile(outputLocation, codec='libx264', audio_codec='aac')

['1.mp4', '2.mp4', '3.mp4', '4.mp4']
['1.mp4', '2.mp4', '3.mp4', '4.mp4']
THERE LIVES WITHIN THE VERY FLAME OF LOVE... A KIND OF WICK OR SNUFF THAT WILL ABATE IT.
abate
Moviepy - Building video /content/drive/MyDrive/greWords/mergedVideos/abate/1.mp4.
MoviePy - Writing audio in 1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/greWords/mergedVideos/abate/1.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/greWords/mergedVideos/abate/1.mp4
CONSUMES IT WITH A BURNING FLAME THAT DOES NOT ABATE.
abate
Moviepy - Building video /content/drive/MyDrive/greWords/mergedVideos/abate/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/greWords/mergedVideos/abate/2.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/greWords/mergedVideos/abate/2.mp4
YOU BASICALLY WANT TO TRY TO EITHER AGGRAVATE THE DEMON OR ABATE IT.
abate
Moviepy - Building video /content/drive/MyDrive/greWords/mergedVideos/abate/3.mp4.
MoviePy - Writing audio in 3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/greWords/mergedVideos/abate/3.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/greWords/mergedVideos/abate/3.mp4
L HAVE UNLEASHED A RAGING SHIT STORM OF EPIC PROPORTIONS ON THE BOARD OF TRUSTEES OF THAT PISSANT SCHOOL THAT WILL NOT ABATE UNTIL THOSE GIRLS SEEK ENROLLMENT ELSEWHERE.
abate
Moviepy - Building video /content/drive/MyDrive/greWords/mergedVideos/abate/4.mp4.
MoviePy - Writing audio in 4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/greWords/mergedVideos/abate/4.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/greWords/mergedVideos/abate/4.mp4


In [88]:
folders = [item for item in os.listdir(outputVideosDirectory) if os.path.isdir(os.path.join(outputVideosDirectory, item)) and not item.startswith(".")]

for folder in folders:
    outputLocation = outputVideosDirectory +"/"+ folder
    myFiles = [file for file in os.listdir(outputLocation) if file.endswith(videoExtension)]
    print(myFiles)
    videoClips = []
    for videoFile in myFiles:
        fileName = videoFile.replace(videoExtension,"")
        videoLocation = outputLocation +"/"+ fileName + videoExtension
        videoClips.append(VideoFileClip(videoLocation, target_resolution=(1080, 1920)))

    finalVideo = concatenate_videoclips(videoClips)  # method="chain" by default. This is fine because all clips are the same size
    outputFile = outputLocation +"/"+ folder + videoExtension
    finalVideo.write_videofile(outputFile, codec="libx264", audio_codec="aac")
    print(f"Gemerated {folder} video")

['1.mp4', '2.mp4', '3.mp4', '4.mp4']
Moviepy - Building video /content/drive/MyDrive/greWords/mergedVideos/abate/abate.mp4.
MoviePy - Writing audio in abateTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/greWords/mergedVideos/abate/abate.mp4



t:  29%|██▉       | 183/629 [00:12<00:30, 14.68it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file /content/drive/MyDrive/greWords/mergedVideos/abate/1.mp4, 6220800 bytes wanted but 0 bytes read,at frame index 175 (out of a total 176 frames), at time 7.30/7.36 sec. Using the last valid frame instead.
  warnings.warn(
t:  42%|████▏     | 263/629 [00:20<00:35, 10.37it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file /content/drive/MyDrive/greWords/mergedVideos/abate/2.mp4, 6220800 bytes wanted but 0 bytes read,at frame index 79 (out of a total 80 frames), at time 3.16/3.20 sec. Using the last valid frame instead.
  warnings.warn(
t:  65%|██████▍   | 407/629 [00:34<00:21, 10.38it/s, now=None]/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:154: UserWarning: In file /content/drive/MyDrive/greWords/mergedVideos/abate/3.mp4, 6220800 bytes want

Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/greWords/mergedVideos/abate/abate.mp4
Gemerated abate video


In [87]:
folders = [item for item in os.listdir(outputVideosDirectory) if os.path.isdir(os.path.join(outputVideosDirectory, item)) and not item.startswith(".")]

for folder in folders:
    outputLocation = outputVideosDirectory +"/"+ folder
    myFiles = [file for file in os.listdir(outputLocation) if file.endswith(videoExtension)]
    allFiles = []
    for files in myFiles:
        allFiles.append(outputLocation +"/"+ files)
    # print(allFiles)

    outputFile = outputLocation +"/"+ folder + videoExtension
    ffmpeg_command = f"ffmpeg -y -f concat -safe 0 -i {allFiles} -c:v h264_nvenc -preset 10 {outputFile}"
    os.system(ffmpeg_command)
    print(1)

1
1
